In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

from modules.mail import send_checks_mail

import sys
from config.variables import user

In [2]:
training_code = 'SGSO'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5479,99999913,PIAZZA,CMT,IB,GRU
5480,99999916,GALENO CABRAL,CMT,IB,GRU
5481,99999917,EDWARD,CMT,IB,GRU
5482,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude
training = pd.read_csv(f'data/neolude/{training_code}.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
2011,00000000,SGSO TRIPULANTES,Em andamento,"0,00",NaT
2001,00000020,SGSO TRIPULANTES,Concluído,"7,00",2020-08-08 19:32:01
1962,00000021,SGSO TRIPULANTES,Concluído,"8,00",2020-12-04 23:07:57
1808,00000022,SGSO TRIPULANTES,Concluído,"9,00",2020-08-03 20:23:54
1697,00000024,SGSO TRIPULANTES,Concluído,"8,00",2020-08-04 16:25:38
...,...,...,...,...,...
139,00044964,SGSO TRIPULANTES,Concluído,"8,00",2020-10-14 18:32:36
1495,00044965,SGSO TRIPULANTES,Concluído,"9,00",2020-10-11 22:59:24
1412,00044976,SGSO TRIPULANTES,Concluído,"7,00",2020-10-01 23:26:13
766,00045083,SGSO TRIPULANTES,Concluído,"10,00",2020-10-12 15:52:57


In [6]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...


In [7]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
98905,00002976,INSS
98923,00034570,INSS
98925,00035529,INSS
98930,00013481,INSS
98935,00021086,INSS
...,...,...
100249,00032174,INSS
100251,00035516,INSS
100260,00022936,INSS
100268,00024216,INSS


In [8]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
98901,00001589,LIC-SREM
98904,00002846,LIC-SREM
98921,00032713,LIC-SREM
98924,00035305,LIC-SREM
98946,00011808,LIC-SREM
...,...,...
100245,00046272,LIC-SREM
100246,00046278,LIC-SREM
100247,00046284,LIC-SREM
100248,00046306,LIC-SREM


In [9]:
rosters = rosters[rosters['code'].isin(funcs.training_filter(training_code))]
rosters = rosters.sort_values(by=['cif','dep'])
rosters = rosters[['cif', 'code', 'fr', 'dep']]
rosters = rosters.drop_duplicates(subset = ['cif'], keep = 'last')
rosters

,cif,code,fr,dep
149472,00000020,C-SGSO-ON,CGH,2020-08-28 16:30:00
66978,00000021,C-SGSO-ON,CGH,2020-12-11 16:15:00
67495,00000022,C-SGSO-ON,CGH,2020-08-13 16:30:00
67478,00000024,C-SGSO-ON,CGH,2020-08-13 16:30:00
32173,00000026,C-SGSO-ON,CGH,2020-10-06 16:30:00
...,...,...,...,...
31534,00046079,C-SGSO,CGH,2020-03-05 11:15:00
31466,00046080,C-SGSO,CGH,2020-03-05 11:15:00
31503,00046081,C-SGSO,CGH,2020-03-05 11:15:00
31461,00046082,C-SGSO,CGH,2020-03-05 11:15:00


In [10]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
68,00033697,SGSO,2022-02-28
70,00001463,SGSO,2022-07-31
71,00010756,SGSO,2022-10-31
75,00033695,SGSO,2022-02-28
76,00024237,SGSO,2022-05-31
...,...,...,...
54830,00045113,SGSO,2021-11-30
54831,00011067,SGSO,2021-11-30
54868,00001066,SGSO,2021-10-31
54869,00005686,SGSO,2021-11-30


In [11]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(rosters[['cif', 'code', 'dep']], on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr
0,00000020,BORIO,CMT,IBX,CGH,SGSO,2022-08-31,SGSO TRIPULANTES,Concluído,"7,00",2020-08-08 19:32:01,C-SGSO-ON,2020-08-28 16:30:00,NaN,NaN
1,00000021,GRABLER,CMT,IBX,GRU,SGSO,2022-12-31,SGSO TRIPULANTES,Concluído,"8,00",2020-12-04 23:07:57,C-SGSO-ON,2020-12-11 16:15:00,NaN,NaN
2,00000022,DANTAS,CMT,IBX,CGH,SGSO,2022-08-31,SGSO TRIPULANTES,Concluído,"9,00",2020-08-03 20:23:54,C-SGSO-ON,2020-08-13 16:30:00,NaN,NaN
3,00000024,BORBA,CMT,NaN,GRU,SGSO,2022-08-31,SGSO TRIPULANTES,Concluído,"8,00",2020-08-04 16:25:38,C-SGSO-ON,2020-08-13 16:30:00,NaN,NaN
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH,SGSO,2022-10-31,SGSO TRIPULANTES,Concluído,"7,00",2020-10-18 11:50:05,C-SGSO-ON,2020-10-06 16:30:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,99999913,PIAZZA,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2030,99999916,GALENO CABRAL,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2031,99999917,EDWARD,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2032,99999918,CESIDIO,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [12]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

validation['dep'] = validation['dep'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['dep'] = pd.to_datetime(validation['dep'], format='%Y-%m-%d')
validation['dep'] = validation['dep'].dt.floor('D')

validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation['tempo_de_conclusao (dias)'] = pd.to_datetime(dt.date(dt.now())) - validation['data_conclusao']

validation['tempo_de_conclusao (dias)'] = validation['tempo_de_conclusao (dias)'].dt.days

validation = validation[validation['dep'] < (pd.to_datetime(dt.date(dt.now())))]
validation['atualizar'] = (validation['status'] == 'Concluído') & (validation['tempo_de_conclusao (dias)'] < 90)

validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar
153,00001593,MARCIO MIGUEIS,CMT,NaN,CGH,SGSO,2018-02-28,SGSO TRIPULANTES,Em andamento,"0,00",2035-12-31,C-SGSO-ON,2020-06-03,NaN,LIC-SREM,-5462,False
342,00003090,IVAN,CMT,NaN,CGH,SGSO,2016-10-31,SGSO TRIPULANTES,Em andamento,"0,00",2035-12-31,C-SGSO-ON,2020-12-22,NaN,NaN,-5462,False
922,00012101,MARCOS GONZAGA,CMT,NaN,BSB,SGSO,2021-01-31,SGSO TRIPULANTES,Em andamento,"0,00",2035-12-31,C-SGSO-ON,2021-01-04,NaN,NaN,-5462,False
1533,00035464,DENIO PACHECO,COP,NaN,CGH,SGSO,2021-01-31,SGSO TRIPULANTES,Em andamento,"0,00",2035-12-31,C-SGSO-ON,2021-01-13,NaN,NaN,-5462,False
1610,00041650,CLAUDIO CINTRA,CMT,NaN,GIG,SGSO,2020-12-29,SGSO TRIPULANTES,Em andamento,"0,00",2035-12-31,C-SGSO-ON,2020-06-09,INSS,NaN,-5462,False
1616,00041776,FILIPE VASCONCELLOS,COP,NaN,GIG,SGSO,2021-01-28,SGSO TRIPULANTES,Em andamento,"0,00",2035-12-31,C-SGSO-ON,2021-01-14,NaN,NaN,-5462,False


In [13]:
validation.to_csv(f'./data/output/SGSO.csv', index=False, sep=';', encoding='utf-8-sig')

In [15]:
# Arquivo de import
# fill table
df = validation[validation['atualizar']]

df

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar


In [16]:
df_ = df[['cif', 'checkTypeCode', 'data_conclusao']]

# fill table

data = df_.copy()

data['update'] = 'U'
data['id'] = df_['cif'].astype(str).str.zfill(8)
data['type'] = 'SGSO'
data['issue'] = df_['data_conclusao'].dt.strftime('%Y%m%d') + '0000'
data['expiry'] = ((df_['data_conclusao'] + \
                   pd.offsets.MonthEnd(0)) + \
                   pd.offsets.DateOffset(years=2)).dt.strftime('%Y%m%d') + '0000'

data['extended'] = data['expiry']
data['failed'] = 'N'
data['do_not_plan_before'] = data['expiry']
data['zero'] = '0'
data['remark'] = ''
data['check_comment'] = ''
data['confirmed'] = 'Y'
data['paper_recieved'] = 'Y'

# remove fields from original file
cols = [
    'update',
    'cif',
    'type',
    'issue',
    'expiry',
    'extended',
    'failed',
    'do_not_plan_before',
    'zero',
    'remark',
    'check_comment',
    'confirmed',
    'paper_recieved'  
]

data = data[cols].copy()

data.to_csv('./data/output/crm_import_sgso.dat', sep='|', header=None, index=False, line_terminator='\r\n')

data

,update,cif,type,issue,expiry,extended,failed,do_not_plan_before,zero,remark,check_comment,confirmed,paper_recieved


In [17]:
out_email = validation.copy()

out_email = out_email[~out_email['inss'].isin(['INSS'])]
out_email = out_email[~out_email['lnr'].isin(['LIC-SREM'])]

out_email = out_email[['cif',
                       'guerra',
                       'funcao',
                       'base',
                       'checkTypeCode',
                       'expiryDt', 
                       'status', 
                       'nota', 
                       'data_conclusao',
                       'code',
                       'dep',
                       'tempo_de_conclusao (dias)',
                       'atualizar']]

out_email['expiryDt'] = out_email['expiryDt'].dt.strftime('%d/%m/%Y')
out_email['data_conclusao'] = out_email['data_conclusao'].dt.strftime('%d/%m/%Y')
out_email['dep'] = out_email['dep'].dt.strftime('%d/%m/%Y')
out_email['status'] = out_email['status'].fillna('')
out_email['nota'] = out_email['nota'].fillna('')

In [18]:
send_checks_mail(out_email, user['mail'], user['password'], user['emails'])